<a href="https://colab.research.google.com/github/salonitripathi/Yelp-Project/blob/master/Copy_of_FINAL_yelp_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import pandas as pd
import collections
import scipy.sparse as sp
import matplotlib.pyplot as plt
import itertools
import numpy as np
import operator
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Flatten, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

#random number generator 
np.random.seed(0)

MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
VALIDATION_SPLIT = 0.2

In [0]:
df_reviews = pd.read_csv('yelp_academic_dataset_review.csv', nrows=500000)

df_reviews['len'] = df_reviews.text.str.len()
df_reviews = df_reviews[df_reviews['len'].between(10, 4000)]


##creating new dataframes for the individual star ratings 
one_star = df_reviews.loc[df_reviews['stars'] == 1]
two_star = df_reviews.loc[df_reviews['stars'] == 2]
three_star = df_reviews.loc[df_reviews['stars'] == 3]
four_star = df_reviews.loc[df_reviews['stars'] == 4]
five_star = df_reviews.loc[df_reviews['stars'] == 5]

##before balancing, will return number of 1,2,4,5 stars 
print(len(one_star))
print(len(two_star))
print(len(three_star))
print(len(four_star))
print(len(five_star))



##balancing --> concatenated a new dataframe with equivalent number of reviews per star 
#returns 5 diff dataframes 
#taking the top 45,000 rows in each star column 

rows = 45000

one_star_balanced = one_star.head(rows)
two_star_balanced = two_star.head(rows)
three_star_balanced = three_star.head(rows)
four_star_balanced = four_star.head(rows)
five_star_balanced = five_star.head(rows)


frames = [one_star_balanced, two_star_balanced, three_star_balanced, four_star_balanced, five_star_balanced]

df_reviews = pd.concat(frames)

#overwrites the original df_reviews into the concatentaed dataframe 

#all the stars put together, but a fixed amount for each one 

#our stars are all in order, so we need to do a shuffle.  


df_reviews = df_reviews.sample(frac=1)

df_reviews 



51174
47694
77905
157251
163804


,user_id,review_id,text,votes.cool,business_id,votes.funny,stars,date,type,votes.useful,len
386475,wY_v5dnDxtvdCClXZWUk3w,UAxQPqe3J0TMwWEtBIgWqw,"Food was ok. Staff was very unhelpful, wouldn'...",0,kAXwEMkpcKMObw45QJGqnw,0,1,2013-09-07,review,0,257
15606,Cd95wUD6uptKdGd3Y3iAyQ,R8BbKRb3QiqQRBl6Wn3AyA,"Nice place, nice guys. They recommended the t...",0,jYyfLGMURG2pQ6k80R4eZA,0,4,2013-10-21,review,1,114
138980,o9jlLuvoxJ3WyaJ12dtS2Q,zSQg8kzO5IzLmvgjLaRKag,My husband purchased a Salon gift package for ...,0,xGiFwao6wwE_ZFYlZbhnqw,0,4,2011-02-06,review,1,493
103675,tjZGvtTG8us5g0QXQ2r3Ow,ce96TyG0ypSgAjGhHkOIXA,I thought the place was alright. The hotel sta...,1,TKE1cyjs9q2PoqMXqEV9Eg,0,3,2011-03-01,review,2,1711
211848,E7Fq8D4Cen2vr0VU6tXfNw,3ZUp-KlEK3yjmaGLepfP3Q,I just stayed four nights and let me tell you ...,0,G0nUhYF2q4tnoOPwCjXLig,0,1,2010-03-04,review,1,640
36432,eg2vtwnEH8aq3csdRUBRKg,evaK5Z1mrPqAcBAICQSTxw,I usually like Hilton chain hotels which is wh...,0,KnRRMCqQQqlyMphif6cTaA,0,3,2014-06-23,review,0,969
33166,QAtL9roCZeqQlLDB-Ara7g,yD7RUDYfUF__C2zn56pG6A,Great location and selection. Libraries are th...,1,NtRs2Xd64mRLe6jGUTXziw,0,5,2014-06-03,review,0,205
72255,KUgw3Us268hflAia7nAPuA,d5WSQ7BN0PS2S_V9eJXk2g,"Plus:\nVery nice, large, modern, comfortable r...",0,vGeat2M50_z7Qumsq0A-lA,0,4,2012-02-22,review,0,812
211827,p2UKiLd0jVx17b4nVNOeKg,OB-UTND3CQOpHCoVxtWq9g,Ok Excalibur isn't a great hotel. It isn't. \n...,2,G0nUhYF2q4tnoOPwCjXLig,0,2,2010-01-21,review,1,470
239865,afp1zk8E0leRe0wIvxBMsw,U9LU-hrwOFAGsNJCM1PMwQ,You know it's bad we when the food gives you a...,0,uSbUk-3UcgzTPpzyL9dgJQ,0,2,2014-01-26,review,0,630


In [0]:
print(df_reviews.iloc[0])
print(df_reviews.shape)

user_id                                    wY_v5dnDxtvdCClXZWUk3w
review_id                                  UAxQPqe3J0TMwWEtBIgWqw
text            Food was ok. Staff was very unhelpful, wouldn'...
votes.cool                                                      0
business_id                                kAXwEMkpcKMObw45QJGqnw
votes.funny                                                     0
stars                                                           1
date                                                   2013-09-07
type                                                       review
votes.useful                                                    0
len                                                           257
Name: 386475, dtype: object
(225000, 11)


In [0]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(df_reviews.text.tolist())

WORD_INDEX_SORTED = sorted(tokenizer.word_index.items(), key=operator.itemgetter(1))

seqs = tokenizer.texts_to_sequences(df_reviews.text.values)
X = pad_sequences(seqs, maxlen=MAX_SEQUENCE_LENGTH)
print(X.shape)

Y = df_reviews.stars.values.astype(int)
Y_cat = to_categorical(Y)
print(Y_cat)

assert X.shape[0] == Y.shape[0]  
X_train, X_test, y_train, y_test = train_test_split(X, Y_cat, test_size=0.2, random_state=9)
num_words = min(MAX_NUM_WORDS, len(WORD_INDEX_SORTED)) + 1

print(num_words)
print(WORD_INDEX_SORTED)
print(df_reviews.shape)

(225000, 1000)
[[0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 ...
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
20001
[('the', 1), ('and', 2), ('i', 3), ('a', 4), ('to', 5), ('was', 6), ('of', 7), ('it', 8), ('for', 9), ('in', 10), ('is', 11), ('that', 12), ('my', 13), ('but', 14), ('this', 15), ('we', 16), ('they', 17), ('you', 18), ('with', 19), ('on', 20), ('not', 21), ('have', 22), ('at', 23), ('had', 24), ('so', 25), ('were', 26), ('are', 27), ('food', 28), ('place', 29), ('good', 30), ('there', 31), ('be', 32), ('me', 33), ('as', 34), ('if', 35), ('like', 36), ('here', 37), ('all', 38), ('just', 39), ('out', 40), ('get', 41), ('one', 42), ('our', 43), ('very', 44), ('when', 45), ('or', 46), ('from', 47), ('time', 48), ('up', 49), ('service', 50), ('great', 51), ('their', 52), ('would', 53), ('go', 54), ('about', 55), ("it's", 56), ('no', 57), ('back', 58), ('what', 59), ('an', 60), ('really', 61), ('some', 62), ('been', 63), ('your', 64), ('only

In [0]:
model = Sequential()

model.add(Embedding(input_dim=num_words, 
                    output_dim=100, 
                    input_length=MAX_SEQUENCE_LENGTH))

model.add(GRU(100, return_sequences=True)) #100 nodes in each GRU, will drop out 20% of the nodes in the previous layer
#dont wanna be overly dependent on specific nodes so we're preventing model from overfitting


model.add(Dropout(0.2)) #dropout is a way of preventing overfitting 
model.add(GRU(100))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))

#last layer we have 6 nodes for each category 0,1,2,3,4,5
#want to represent each category as a probablity, essentailly just normalizing all values add up to 1 


model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

print(model.summary())
model.fit(X_train, y_train, 
          epochs=4, 
          batch_size=128, 
          validation_data=(X_test, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 100)         2000100   
_________________________________________________________________
gru_3 (GRU)                  (None, 1000, 100)         60300     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000, 100)         0         
_________________________________________________________________
gru_4 (GRU)                  (None, 100)               60300     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 606       
Total params: 2,121,306
Trainable params: 2,121,306
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.save("yelp.model")

In [0]:
##insert any review 

#review = "My recent visit with this location was HORRIBLE....If i could give this Starbucks-10 I WOULD.. I went to order my favorite hibiscus drink and also my husband FREE birthday DRINK...mind you my husband has been a Starbucks member for over 2 years or so...the coupon didn't show up on his app so we tried to show him on his birthday his id that it was actually his birthday the girl simply replied by being snappy"

review = "This starbucks is just okay. I think the location is too small for the volume of people they get. Since its right across from Columbia University, it can get pretty packed sometimes. There is ALWAYS a line during the morning, so I recommend ordering through the app. Even then, the wait can be long. I had to wait about 10 minutes today for a caramel macchiato despite ordering in advance."

seq=tokenizer.texts_to_sequences([review])

seq_pad = pad_sequences(seq,maxlen=1000)

print("number of stars:")
model.predict_classes(seq_pad)[0]

number of stars:


3